#4.5. Fes una funció, anomenada dna, basada en l’algorisme de Levenshtein, que busqui dins d’una seqüència genètica una cadena genètica passada per paràmetre. Aquesta funció ha de retornar la lı́nia del fitxer on comença la cadena més semblant i la distància entre la cadena d’entrada i la cadena més semblant.

In [51]:
def levensthein(patro, text, dlt = 2, insr = 2, subs = 1):
    if len(patro) > len(text): 
        patro, second = text, patro
    if len(text) == 0: 
        return len(patro)
    first_length = len(patro) + 1
    second_length = len(text) + 1

    distance_matrix = [[0] * second_length for x in range(first_length)]
    operacions_matrix = [[0] * second_length for x in range(first_length)]

    #inicialitzo 1ra fila i columna de la matriu d'operacions tal i com s'indica a les diapositives de teoria
    for i in range(first_length):
        operacions_matrix[i][0] = 'D'
    for i in range(second_length):
        operacions_matrix[0][i] = 'I'
    operacions_matrix[0][0] = 'S'

    #omplo la matriu de distàncies
    for i in range(first_length): 
        distance_matrix[i][0] = i
    for j in range(second_length): 
        distance_matrix[0][j] = 0 
    for i in range(1, first_length):
        for j in range(1, second_length):
            deletion = distance_matrix[i-1][j] + dlt  
            insertion = distance_matrix[i][j-1] + insr
            substitution = distance_matrix[i-1][j-1] 
            if patro[i-1] != text[j-1]: 
                substitution += subs  
            distance_matrix[i][j] = min(substitution,insertion,deletion)
            
            #omplo la matriu d'operacions
            if distance_matrix[i][j] == substitution:
                if distance_matrix[i-1][j-1] + subs == substitution:
                    operacions_matrix[i][j] = 'S'
                else:
                    operacions_matrix[i][j] = 'C'
            elif distance_matrix[i][j] == insertion:
                operacions_matrix[i][j] = 'I'
            else:
                operacions_matrix[i][j] = 'D'
            
    distancia_minima = min(distance_matrix[first_length-1])
    posicio_min = distance_matrix[first_length-1].index(distancia_minima)

    
    #backtracking
    cami = []
    i = first_length - 1
    j = posicio_min
    while i > 0:
        cami.append(operacions_matrix[i][j])
        if operacions_matrix[i][j] == 'D':
            i = i - 1
        elif operacions_matrix[i][j] == 'I':
            j = j - 1
        else:
            i = i - 1
            j = j - 1

    cami = cami[::-1]
    return distancia_minima, posicio_min - first_length + 1, posicio_min, cami

In [49]:
def dna(patro, fitxer = 'HUMAN-DNA.txt'):
    with open(fitxer,'r') as f:
        text = f.read()
        linies = text.split('\n')
    distancia_min = len(linies[0])
    cami = []
    cami_min = []
    for linia in linies:
        distancia, posicio_inicial, posicio_final, cami = levensthein(patro,linia)
        if distancia < distancia_min:
            distancia_min = distancia
            num_linia = linies.index(linia)
            posicio = (posicio_inicial, posicio_final)
            cami_min = cami
    return num_linia, posicio, distancia_min, cami_min

Tal i com hem comentat al professorat, els asserts que porten una 'I', a tots els alumnes ens surt que hauria de ser una 'D', ja que tal i com ens ha explicat el professor de laboratori el problema, hauria de ser així.

In [50]:
assert dna("CTGGTACCAGCTGTATTAGC") == (728, (11, 31), 6, ['C', 'C', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'S', 'C', 'S', 'S', 'S', 'C', 'C', 'S', 'C', 'C', 'C'])
assert dna("TCGTCATAAACCGCTGTGCC") == (212, (12, 32), 7, ['S', 'C', 'S', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'S', 'S', 'C', 'C', 'S', 'S', 'C', 'S'])
assert dna("TATACAAACGGAGTAGCTGT") == (285, (5, 25), 6, ['C', 'C', 'C', 'C', 'S', 'C', 'S', 'C', 'C', 'S', 'C', 'S', 'S', 'C', 'C', 'C', 'S', 'C', 'C', 'C'])
assert dna("AGGCGTAAGTCTTACGTATA") == (5, (41, 61), 7, ['C', 'S', 'C', 'S', 'S', 'C', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'S', 'S', 'C', 'S', 'C', 'C', 'C'])
assert dna("AACGGCATAGCCTGCAAGAG") == (433, (41, 61), 5, ['C', 'C', 'S', 'C', 'C', 'C', 'C', 'S', 'C', 'S', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'S'])
assert dna("GTGCGTCCACCCTTAATACA") == (196, (41, 61), 6, ['C', 'C', 'C', 'S', 'S', 'C', 'S', 'C', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'S', 'C', 'C', 'S', 'C'])
assert dna("GTCAGCACCGGGATCTGTAT") == (240, (26, 46), 7, ['C', 'S', 'C', 'S', 'S', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'C', 'S', 'C', 'S', 'C', 'C', 'S', 'C'])
assert dna("GAGCCCCGACGTTTTAACGA") == (68, (6, 26), 7, ['S', 'C', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'C', 'S', 'C', 'S', 'C', 'C', 'C', 'S', 'S', 'S', 'C'])
assert dna("CCTCGTAGACAGTACCGAAT") == (448, (30, 50), 6, ['C', 'S', 'C', 'C', 'S', 'C', 'C', 'S', 'C', 'C', 'C', 'S', 'C', 'C', 'C', 'C', 'S', 'S', 'C', 'C'])
assert dna("CGACCAAAGAGCCTGTATCT") == (320, (35, 55), 7, ['S', 'S', 'C', 'S', 'S', 'C', 'C', 'C', 'C', 'S', 'C', 'S', 'C', 'C', 'S', 'C', 'C', 'C', 'C', 'C'])
assert dna("GACACACACTTGGATCTTCG") == (565, (16, 36), 6, ['C', 'S', 'C', 'C', 'C', 'S', 'S', 'C', 'C', 'C', 'C', 'C', 'S', 'S', 'S', 'C', 'C', 'C', 'C', 'C'])
assert dna("CTGGACGTTCATACATAGAC") == (28, (21, 41), 7, ['C', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'C', 'S', 'S', 'S', 'C', 'S', 'C', 'S', 'C', 'C', 'C', 'S'])
assert dna("ACGTTTTACCCCAAAGCCCG") == (753, (4, 24), 7, ['C', 'S', 'S', 'S', 'S', 'C', 'C', 'C', 'C', 'S', 'C', 'C', 'C', 'C', 'C', 'C', 'S', 'C', 'S', 'C'])
assert dna("CGGGTAGAAATCTCCGCTTG") == (361, (30, 50), 6, ['S', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'S', 'S', 'C', 'C', 'S', 'C', 'C', 'S', 'S', 'C', 'C', 'C'])